# Algo Trading- Blankly-Master Template

In this notebook, we will build a backtrader strategy using any trading algorithm


https://blankly.finance/blankly-with-jupyter-notebooks/
To do:
* Make the tearsheet work
* or use the tearsheet of omegaUI
* Put Teststrategy.py in the algos folder

[1. Step 1 : Loading the algorithm](#1)

[2. Step 2 : Setting the backtester](#2)

[3. Step 3 : Getting the backtest output](#3)

In [3]:
!blankly init

Traceback (most recent call last):
  File "C:\Users\tatsa\anaconda3\lib\site-packages\blankly\deployment\new_cli.py", line 526, in main
    args.func(args)
  File "C:\Users\tatsa\anaconda3\lib\site-packages\blankly\deployment\new_cli.py", line 184, in blankly_init
    and not confirm('This directory is not empty. Would you like to continue anyways?').unsafe_ask():
  File "C:\Users\tatsa\anaconda3\lib\site-packages\questionary\prompts\confirm.py", line 127, in confirm
    PromptSession(
  File "C:\Users\tatsa\anaconda3\lib\site-packages\prompt_toolkit\shortcuts\prompt.py", line 468, in __init__
    self.app = self._create_application(editing_mode, erase_when_done)
  File "C:\Users\tatsa\anaconda3\lib\site-packages\prompt_toolkit\shortcuts\prompt.py", line 719, in _create_application
    application: Application[_T] = Application(
  File "C:\Users\tatsa\anaconda3\lib\site-packages\prompt_toolkit\application\application.py", line 277, in __init__
    self.output = output or session.output

<a id='1'></a>
## 1. Step 1 : Loading the algorithm
In this step we load a sample TestStrategy which is created separately in a python file.

In [5]:
from blankly import Strategy, StrategyState, Alpaca
from blankly.indicators import sma
from bokeh.io import output_notebook 
# this is to ensure that any plots are displayed in jupyter notebook
output_notebook()

Loading BokehJS ...

In [7]:
#a = Alpaca()

In [ ]:
a.interface.history('NVDA', 5, resolution='1d')

In [ ]:
def init(symbol, state: StrategyState):
    interface: Interface = state.interface
    resolution: float = state.resolution
    variables = state.variables
    # initialize the historical data as a deque to save memory and effiency
    variables['history'] = interface.history(symbol, 800, resolution, return_as='deque')['close']
    variables['has_bought'] = False

In [ ]:
def golden_cross(price, symbol, state: StrategyState):
    interface: Interface = state.interface
    variables = state.variables

    variables['history'].append(price)

    sma100 = sma(variables['history'], period=20)
    
    # match up dimensions so that we can use numpy for difference
    sma50 = sma(variables['history'], period=10)[-len(sma100):]
    diff = sma100 - sma50
    
    slope_sma50 = (sma50[-1] - sma50[-5]) / 5 # get the slope of the last 5 SMA50 Data Points
    prev_diff = diff[-2]
    curr_diff = diff[-1]
    
    is_cross_up = slope_sma50 > 0 and curr_diff >= 0 and prev_diff < 0
    is_cross_down = slope_sma50 < 0 and curr_diff <= 0 and prev_diff > 0
    
    # comparing prev diff with current diff will show a cross
    if is_cross_up and not variables['has_bought']:
        # determine how much we want to buy based on our cash allocation
        interface.market_order(symbol, 'buy', int(interface.cash / price))
        variables['has_bought'] = True
    elif is_cross_down and variables['has_bought']:
        # use strategy.base_asset if on CoinbasePro or Binance
        # truncate here to fix any floating point errors
        interface.market_order(symbol, 'sell', int(interface.account[symbol].available))
        variables['has_bought'] = False

In [ ]:
s = Strategy(a)
s.add_price_event(golden_cross, 'MSFT', resolution='1d', init=init)

<a id='2'></a>
## 2. Step 2 : Setting the backtester
In this step, we set up a backtester, with cash amount, data and strategy name updated.

In [52]:
def backtest():
    #path_dir = os.path.dirname(os.path.realpath(__file__))
    
    cash = 100000
    symbols = ['AAPL','AMZN']
    #start_date = '2018-01-01'
    path_dir = 'Data'
    #strategy = "TestStrategy"
    
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)

    for s in symbols:
            df = pd.read_csv(os.path.join(path_dir, '{}.csv'.format(s)), parse_dates=True, index_col=0)
            data = bt.feeds.PandasData(dataname=df)
            cerebro.adddata(data)
    #data = bt.feeds.PandasData(dataname=yf.download('TSLA', '2018-01-01', '2018-01-10'))
    #cerebro.adddata(data)

    # Strategy
    cerebro.addstrategy(TestStrategy)


    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.SQN, _name='SQN')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    #cerebro.adddata(data)
    # Backtest

    print('Starting Portfolio Value: ', cerebro.broker.getvalue())
    plt.rcParams['figure.figsize']=[10,6]
    plt.rcParams["font.size"]="12"
    # Run over everything
    results = cerebro.run()
    pnl = cerebro.broker.getvalue() - cash
    #cerebro.plot()
    # Print out the final result
    print('Final Portfolio Value: ' , cerebro.broker.getvalue())    
    strat = results[0]
    pyfoliozer = strat.analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    return returns, positions, transactions, gross_lev, pnl
    #return results[0], pnl

In [53]:
def test_results():
    returns, positions, transactions, gross_lev, pnl = backtest()
    # returns, pnl = backtest()

In [54]:
test_results()

Starting Portfolio Value:  100000
Final Portfolio Value:  94485.26799999991


<a id='3'></a>
## 3. Step 3 : Getting the backtest output
In this step, we set up a backtester, with cash amount, data and strategy name updated.

In [55]:
returns, positions, transactions, gross_lev, pnl = backtest()

Starting Portfolio Value:  100000
Final Portfolio Value:  94485.26799999991


In [64]:
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    gross_lev=gross_lev,
    live_start_date='2005-05-01',  # This date is sample specific
    round_trips=True)


TypeError: create_full_tear_sheet() got an unexpected keyword argument 'gross_lev'